In [1]:
!pip install huggingface_hub > /dev/null

In [2]:
!pip install llcuda  > /dev/null

In [3]:
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('HF_TOKEN'))

In [4]:
import os
import sys
from pathlib import Path

In [5]:
import llcuda
print("llcuda imported successfully – setup complete if first run!")

🎯 llcuda First-Time Setup

🎮 GPU Detected: Tesla T4 (Compute 7.5)
🌐 Platform: Colab

📦 Selecting optimized binaries for your GPU...
  Detected modern GPU (CC 7.5), using T4 binaries
   Selected: llcuda-binaries-cuda12-t4.tar.gz

📥 Downloading optimized binaries from GitHub...
   URL: https://github.com/waqasm86/llcuda/releases/download/v1.2.2/llcuda-binaries-cuda12-t4.tar.gz
   This is a one-time download (~270 MB)


📦 Extracting llcuda-binaries-cuda12-t4.tar.gz...
Found 25 files in archive
Extracted 25 files to /root/.cache/llcuda/extract
✅ Extraction complete!
📂 Installing binaries and libraries...
⚠️  Expected structure not found, searching for binaries...
  ✅ Found and copied llama-server
  ✅ Found and copied 18 libraries
✅ Binaries installed successfully!

✅ llcuda Setup Complete!

You can now use llcuda:

  import llcuda
  engine = llcuda.InferenceEngine()
  engine.load_model('gemma-3-1b-Q4_K_M')  # Downloads model on first use
  result = engine.infer('What is AI?')

llcuda impor

In [ ]:
#cell not executed yet.
import os
from pathlib import Path

# Directory containing the .so libraries (libmtmd.so.0, libggml-cuda.so, etc.)
lib_dir = Path('/usr/local/lib/python3.12/dist-packages/llcuda/lib')

if lib_dir.exists():
    os.environ['LD_LIBRARY_PATH'] = str(lib_dir) + ':' + os.environ.get('LD_LIBRARY_PATH', '')
    print("✓ LD_LIBRARY_PATH set to include llcuda libraries")
    print(f"LD_LIBRARY_PATH now includes: {lib_dir}")
else:
    print("Warning: Library directory not found!")

In [6]:
compat = llcuda.check_gpu_compatibility()
print(f"Platform: {compat['platform']}")
print(f"GPU: {compat['gpu_name']}")
print(f"Compute Capability: {compat['compute_capability']}")
print(f"Compatible: {compat['compatible']}")

assert compat['compatible'], "GPU not compatible!"

Platform: colab
GPU: Tesla T4
Compute Capability: 7.5
Compatible: True


In [7]:
engine = llcuda.InferenceEngine()

engine.load_model(
    "unsloth/gemma-3-1b-it-GGUF:gemma-3-1b-it-Q4_K_M.gguf",
     silent=True,
    gpu_layers=26,
    ctx_size=2048
)

print("Model loaded successfully! Server is running.")

Loading model: unsloth/gemma-3-1b-it-GGUF:gemma-3-1b-it-Q4_K_M.gguf

Repository: unsloth/gemma-3-1b-it-GGUF
File: gemma-3-1b-it-Q4_K_M.gguf
Cache location: /usr/local/lib/python3.12/dist-packages/llcuda/models/gemma-3-1b-it-Q4_K_M.gguf

Download this model? [Y/n]: y



/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


gemma-3-1b-it-Q4_K_M.gguf:   0%|          | 0.00/806M [00:00<?, ?B/s]

✓ Model downloaded: gemma-3-1b-it-Q4_K_M.gguf

Starting llama-server...
GPU Check:
  Platform: colab
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llcuda/binaries/cuda12/llama-server
  Model: gemma-3-1b-it-Q4_K_M.gguf
  GPU Layers: 26
  Context Size: 2048
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready...... ✓ Ready in 3.0s

✓ Model loaded and ready for inference
  Server: http://127.0.0.1:8090
  GPU Layers: 26
  Context Size: 2048
Model loaded successfully! Server is running.


In [8]:
#cell not executed yet.
prompt = "Explain quantum entanglement in simple terms."

result = engine.infer(prompt, max_tokens=300, temperature=0.7)

print("Response:")
print(result.text.strip())
print("\n--- Metrics ---")
print(f"Tokens generated: {result.tokens_generated}")
print(f"Speed: {result.tokens_per_sec:.1f} tokens/second")

Response:
Okay, let's tackle quantum entanglement. It's a mind-bending concept, but we can break it down.

**The Basic Idea:**

Imagine you have two coins. You flip them at the same time, but you don’t look at them.  You put each coin in a separate box and send one box to Alice and one box to Bob.  

*   **Normal Coins:**  When Alice opens her box and finds a heads, she knows Bob *must* have a tails.  The outcome of one instantly determines the other.

*   **Quantum Entangled Coins:** Now, imagine these aren't regular coins.  These coins are linked in a bizarre way.  Until *either* Alice or Bob opens their box, *neither* coin is definitively heads or tails.  They're in a fuzzy state of *both* at the same time. The moment Alice opens her box and sees heads, Bob *instantly* knows his coin is tails, even if he hasn't opened his box yet.

**Here's a breakdown of the key points:**

1.  **Linked Fate:** Quantum entanglement links two or more particles (like electrons or photons) in such a wa

In [9]:
#cell not executed yet.
def chat():
    print("Chat with Gemma 3 1B (type 'exit' to quit)\n")
    history = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break
        full_prompt = ""
        for msg in history:
            full_prompt += f"<start_of_turn>{msg['role']}\n{msg['content']}<end_of_turn>\n"
        full_prompt += f"<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"

        result = engine.infer(full_prompt, max_tokens=400)
        response = result.text.strip()
        print(f"Assistant: {response}\n")
        history.append({"role": "user", "content": user_input})
        history.append({"role": "model", "content": response})

# chat()

In [10]:
chat()

Chat with Gemma 3 1B (type 'exit' to quit)

You: What llm is this?
Assistant: I am Gemma, a large language model created by the Gemma team at Google DeepMind. 

I am an open-weights model, which means I am widely available for public use! 

Is there anything you’d like to know about me or how I work?

You: Explain about your own technical details.
Assistant: Okay, let’s dive into my technical details! It’s a surprisingly complex thing, but here’s a breakdown in a way that hopefully makes sense:

**1. Foundation: Transformer Architecture**

*   I’m built on the Transformer architecture, which revolutionized the field of natural language processing.  Previous models relied heavily on recurrent neural networks (RNNs), but Transformers are better at handling long-range dependencies in text – meaning they can understand context across long sentences or even paragraphs.
*   **Key Components:**
    *   **Self-Attention:** This is the core of the Transformer. It allows me to weigh the importan

KeyboardInterrupt: Interrupted by user